In [31]:
from __future__ import print_function
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import itertools
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from scikitplot.metrics import plot_roc_curve
from scikitplot.decomposition import plot_pca_component_variance
from scikitplot.estimators import plot_feature_importances
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, VotingClassifier
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

In [6]:
# set up display area to show dataframe in jupyter qtconsole
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [7]:
train_dir = 'train/'
test_dir = 'test/'

# Load and analyse data

In this step we will analyse different columns of dataset. All the columns will go through feature processing pipeline next.

In [71]:
train_data = dict()
new_name = ['pporder', 'ppdriver', 'prestock', 'poststock']
for i, name in enumerate(os.listdir(train_dir)):
    train_data[new_name[i]] = pd.read_csv(os.path.join(train_dir, name))

In [72]:
train_data['pporder'].head(3)

,created_at,city,res_id,pickup_locality,pickup_latitude,pickup_longitude,drop_locality,drop_latitude,drop_longitude,total_amount,driver_id,shipment_creation_time,device_acknowledge_at,device_acknowledge_latitude,device_acknowledge_longitude,device_acknowledge_location_update_time,driver_assigned_at,driver_assigned_latitude,driver_assigned_longitude,driver_assigned_location_update_time,reached_shop_at,reached_shop_latitude,reached_shop_longitude,reached_shop_location_update_time,shipped_at,shipped_latitude,shipped_longitude,shipped_location_update_time,reached_customer_at,reached_customer_latitude,reached_customer_longitude,reached_customer_location_update_time,complete_at,complete_latitude,pickup_to_drop_location_km,pickup_to_drop_location_aerial_km,state
0,01-01-2018 03:52,Bangalore,18369812.0,HSR Layout,12.913377,77.636743,HSR Layout,12.918794,77.643636,234.0,19858.0,01-01-2018 03:57,01-01-2018 03:57,12.913377,77.636743,57:49.9,01-01-2018 03:58,12.911163,77.645047,58:01.7,01-01-2018 04:07,12.912745,77.636802,07:26.7,01-01-2018 04:20,12.915067,77.638542,20:22.9,01-01-2018 04:23,12.919423,77.642763,23:29.5,01-01-2018 04:31,12.918759,1.351,0.973827,COMPLETE
1,01-01-2018 04:17,Bangalore,18432984.0,Koramangala,12.932813,77.614281,BTM Layout,12.908013,77.612200,596.2,NaN,01-01-2018 04:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.061,2.769349,CANCELLED
2,01-01-2018 04:39,Bangalore,18527596.0,Koramangala,12.934553,77.617152,Koramangala,12.929307,77.622008,185.0,NaN,01-01-2018 04:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,CANCELLED


In [73]:
train_data['pporder'].loc[lambda df: df['res_id'] == 18460060, :]

,created_at,city,res_id,pickup_locality,pickup_latitude,pickup_longitude,drop_locality,drop_latitude,drop_longitude,total_amount,driver_id,shipment_creation_time,device_acknowledge_at,device_acknowledge_latitude,device_acknowledge_longitude,device_acknowledge_location_update_time,driver_assigned_at,driver_assigned_latitude,driver_assigned_longitude,driver_assigned_location_update_time,reached_shop_at,reached_shop_latitude,reached_shop_longitude,reached_shop_location_update_time,shipped_at,shipped_latitude,shipped_longitude,shipped_location_update_time,reached_customer_at,reached_customer_latitude,reached_customer_longitude,reached_customer_location_update_time,complete_at,complete_latitude,pickup_to_drop_location_km,pickup_to_drop_location_aerial_km,state
1146,05-01-2018 15:17,Bangalore,18460060.0,Koramangala,12.952484,77.6228,MG Road,12.972813,77.609122,282.0,15098.0,05-01-2018 15:18,05-01-2018 15:18,12.952484,77.622800,18:07.1,05-01-2018 15:18,12.943728,77.630447,18:07.1,05-01-2018 15:32,12.952517,77.622690,32:38.5,05-01-2018 15:34,12.952600,77.622597,34:06.3,05-01-2018 16:02,12.972973,77.609475,02:13.2,05-01-2018 16:05,12.972813,3.937,2.705520,COMPLETE
2140,04-01-2018 12:39,Bangalore,18460060.0,Koramangala,12.952484,77.6228,Koramangala,12.948848,77.617090,167.0,20608.0,04-01-2018 12:40,04-01-2018 12:40,12.952484,77.622800,40:04.8,04-01-2018 12:40,12.947425,77.621617,40:04.8,04-01-2018 12:55,12.952897,77.622721,55:49.6,04-01-2018 12:56,12.952897,77.622721,55:59.5,04-01-2018 13:14,12.948848,77.617090,14:48.8,04-01-2018 13:14,12.948848,1.056,0.739819,COMPLETE
2152,04-01-2018 07:09,Bangalore,18460060.0,Koramangala,12.952484,77.6228,MG Road,12.966227,77.595784,157.0,15886.0,04-01-2018 07:09,04-01-2018 07:09,12.952484,77.622800,09:48.5,04-01-2018 07:10,12.947361,77.616784,10:26.3,04-01-2018 07:16,12.952555,77.623025,16:53.4,04-01-2018 07:19,12.952660,77.622799,19:57.5,04-01-2018 07:45,12.966123,77.595567,45:10.2,04-01-2018 07:45,12.966227,4.523,3.305365,COMPLETE
2302,07-01-2018 13:53,Bangalore,18460060.0,Koramangala,12.952484,77.6228,MG Road,12.963608,77.598512,189.0,23150.0,07-01-2018 13:54,07-01-2018 15:34,12.963386,77.598789,34:35.7,07-01-2018 15:35,12.961242,77.597877,35:07.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07-01-2018 15:46,12.951907,4.243,2.910735,RETURN
2687,08-01-2018 12:05,Bangalore,18460060.0,Koramangala,12.952484,77.6228,MG Road,12.966128,77.598456,147.0,19357.0,08-01-2018 12:07,08-01-2018 12:07,12.952484,77.622800,07:09.5,08-01-2018 12:07,12.947430,77.617965,07:23.2,08-01-2018 12:54,12.952661,77.622993,54:41.8,08-01-2018 12:54,12.952662,77.622900,54:58.3,08-01-2018 13:20,12.966128,77.598456,20:16.6,08-01-2018 13:20,12.966128,4.249,3.045893,COMPLETE
3036,09-01-2018 07:50,Bangalore,18460060.0,Koramangala,12.952484,77.6228,Ulsoor,12.972618,77.629435,147.0,24545.0,09-01-2018 07:50,09-01-2018 07:51,12.952484,77.622800,50:59.0,09-01-2018 07:51,12.950670,77.623741,51:15.7,09-01-2018 07:59,12.952655,77.622743,59:45.5,09-01-2018 08:00,12.952667,77.622735,00:35.0,09-01-2018 08:24,12.972851,77.629528,24:20.6,09-01-2018 08:25,12.972618,4.197,2.353545,COMPLETE
3551,10-01-2018 14:10,Bangalore,18460060.0,Koramangala,12.952484,77.6228,Koramangala,12.929485,77.610634,140.0,18960.0,10-01-2018 14:12,10-01-2018 14:12,12.952484,77.622800,12:52.1,10-01-2018 14:12,12.954281,77.616201,12:59.1,10-01-2018 14:28,12.952630,77.622686,28:21.3,10-01-2018 14:29,12.952616,77.622979,29:10.3,10-01-2018 14:54,12.929485,77.610634,52:56.6,10-01-2018 14:58,12.929485,4.011,2.879833,COMPLETE
4179,12-01-2018 08:14,Bangalore,18460060.0,Koramangala,12.952484,77.6228,Ulsoor,12.971271,77.630141,147.0,18960.0,12-01-2018 08:15,12-01-2018 08:15,12.952484,77.622800,15:43.1,12-01-2018 08:15,12.954281,77.616201,15:43.1,12-01-2018 08:29,12.952745,77.622593,29:22.4,12-01-2018 08:29,12.952689,77.622666,29:49.3,12-01-2018 09:00,12.971271,77.630141,00:29.7,12-01-2018 09:00,12.971271,3.984,2.237376,COMPLETE
4319,12-01-2018 09:00,Bangalore,18460060.0,Koramangal

In [74]:
train_data['ppdriver'].head(3)

,delivery_driver_id,date,login_time,logout_time,locality_name,city_name,latitude,longitude
0,28,2017-12-31,2018-01-01 05:24:26,2018-01-01 05:24:31,Koramangala,Bangalore,NaN,NaN
1,28,2017-12-31,2018-01-01 05:29:24,2018-01-01 05:56:38,Koramangala,Bangalore,NaN,NaN
2,28,2018-01-01,2018-01-01 06:22:50,2018-01-01 06:27:35,Koramangala,Bangalore,NaN,NaN


In [76]:
train_data['prestock'].head(3)

,geography,time_stamp_utc,dt,Latitude,Longitude,res_id,stockout
0,koramangala,03JAN2018:15:15:12,03JAN2018,12.952484,77.6228,18460060,1
1,koramangala,03JAN2018:15:15:39,03JAN2018,12.952484,77.6228,18460060,1
2,koramangala,03JAN2018:15:18:18,03JAN2018,12.952484,77.6228,18460060,1


In [77]:
train_data['poststock'].head(3)

,geography,time_stamp_utc,dt,Latitude,Longitude,res_id,stockout
0,koramangala,29JAN2018:13:00:24,29JAN2018,12.952484,77.6228,18460060,0
1,koramangala,29JAN2018:13:00:48,29JAN2018,12.952484,77.6228,18460060,0
2,koramangala,29JAN2018:13:02:11,29JAN2018,12.952484,77.6228,18460060,0


# Feature Engineering

We will start cleaning the train data. Initial approach will be to seperate each data and time into seperate columns of time --> hr, min, sec and date --> year, month, week, day

In [78]:
def adddate(df):
    for c in df.columns:
        if '_at' in c or '_creation_time' in c:
            print (c)
            df.Date = pd.to_datetime(df[c], dayfirst=True)
            df[c.split('_')[0]+'_year']  = df.Date.dt.year
            df[c.split('_')[0]+'_month'] = df.Date.dt.month
            df[c.split('_')[0]+'_week']  = df.Date.dt.week
            df[c.split('_')[0]+'_day']   = df.Date.dt.day
            df[c.split('_')[0]+'_hrs']   = df.Date.dt.hour
            df[c.split('_')[0]+'_min']   = df.Date.dt.minute
            df.drop([c], axis=1, inplace=True)
        elif '_update_time' in c:
            print (c)
            df['_'.join(c.split('_', 2)[:2])+'_min'] = df[c].str.split(pat=':', expand=True)[0]
            df['_'.join(c.split('_', 2)[:2])+'_sec'] = df[c].str.split(pat=':', expand=True)[1]
            df.drop([c], axis=1, inplace=True)
            
def adddate_driver(df):
    for c in df.columns:
        if '_time' in c:
            print (c)
            df.Date = pd.to_datetime(df[c], dayfirst=False)
            df[c.split('_')[0]+'_year']  = df.Date.dt.year
            df[c.split('_')[0]+'_month'] = df.Date.dt.month
            df[c.split('_')[0]+'_week']  = df.Date.dt.week
            df[c.split('_')[0]+'_day']   = df.Date.dt.day
            df[c.split('_')[0]+'_hrs']   = df.Date.dt.hour
            df[c.split('_')[0]+'_min']   = df.Date.dt.minute
            df.drop([c], axis=1, inplace=True)
        elif c in ['date']:
            print (c)
            df.Date = pd.to_datetime(df[c], dayfirst=False)
            df[c.split('_')[0]+'_year']  = df.Date.dt.year
            df[c.split('_')[0]+'_month'] = df.Date.dt.month
            df[c.split('_')[0]+'_week']  = df.Date.dt.week
            df[c.split('_')[0]+'_day']   = df.Date.dt.day
            df.drop([c], axis=1, inplace=True)
            
def adddate_pp(df):
    for c in df.columns:
        if c in ['time_stamp_utc']:
            print (c)
            d = df[c].str.split(':', expand=True)
            df.Date = pd.to_datetime(d[0], dayfirst=True, format="%d%b%Y")
            df['_'.join(c.split('_', 2)[:2])+'_year']  = df.Date.dt.year
            df['_'.join(c.split('_', 2)[:2])+'_month'] = df.Date.dt.month
            df['_'.join(c.split('_', 2)[:2])+'_week']  = df.Date.dt.week
            df['_'.join(c.split('_', 2)[:2])+'_day']   = df.Date.dt.day            
            df['_'.join(c.split('_', 2)[:2])+'_hrs'] = df[c].str.split(pat=':', expand=True)[1]
            df['_'.join(c.split('_', 2)[:2])+'_min'] = df[c].str.split(pat=':', expand=True)[2]
            df['_'.join(c.split('_', 2)[:2])+'_sec'] = df[c].str.split(pat=':', expand=True)[3]
            df.drop([c], axis=1, inplace=True)
        elif c in ['dt']:
            print (c)
            df.Date = pd.to_datetime(df[c], dayfirst=True, format="%d%b%Y")
            df[c.split('_')[0]+'_year']  = df.Date.dt.year
            df[c.split('_')[0]+'_month'] = df.Date.dt.month
            df[c.split('_')[0]+'_week']  = df.Date.dt.week
            df[c.split('_')[0]+'_day']   = df.Date.dt.day  
            df.drop([c], axis=1, inplace=True)            

In [79]:
df = train_data['pporder']
adddate(df)
df.head(2)

created_at
shipment_creation_time
device_acknowledge_at
device_acknowledge_location_update_time
driver_assigned_at
driver_assigned_location_update_time
reached_shop_at
reached_shop_location_update_time
shipped_at
shipped_location_update_time
reached_customer_at
reached_customer_location_update_time
complete_at


,city,res_id,pickup_locality,pickup_latitude,pickup_longitude,drop_locality,drop_latitude,drop_longitude,total_amount,driver_id,device_acknowledge_latitude,device_acknowledge_longitude,driver_assigned_latitude,driver_assigned_longitude,reached_shop_latitude,reached_shop_longitude,shipped_latitude,shipped_longitude,reached_customer_latitude,reached_customer_longitude,complete_latitude,pickup_to_drop_location_km,pickup_to_drop_location_aerial_km,state,created_year,created_month,created_week,created_day,created_hrs,created_min,shipment_year,shipment_month,shipment_week,shipment_day,shipment_hrs,shipment_min,device_year,device_month,device_week,device_day,device_hrs,device_min,device_acknowledge_min,device_acknowledge_sec,driver_year,driver_month,driver_week,driver_day,driver_hrs,driver_min,driver_assigned_min,driver_assigned_sec,reached_year,reached_month,reached_week,reached_day,reached_hrs,reached_min,reached_shop_min,reached_shop_sec,shipped_year,shipped_month,shipped_week,shipped_day,shipped_hrs,shipped_min,shipped_location_min,shipped_location_sec,reached_customer_min,reached_customer_sec,complete_year,complete_month,complete_week,complete_day,complete_hrs,complete_min
0,Bangalore,18369812.0,HSR Layout,12.913377,77.636743,HSR Layout,12.918794,77.643636,234.0,19858.0,12.913377,77.636743,12.911163,77.645047,12.912745,77.636802,12.915067,77.638542,12.919423,77.642763,12.918759,1.351,0.973827,COMPLETE,2018,1,1,1,3,52,2018.0,1.0,1.0,1.0,3.0,57.0,2018.0,1.0,1.0,1.0,3.0,57.0,57,49.9,2018.0,1.0,1.0,1.0,3.0,58.0,58,01.7,2018.0,1.0,1.0,1.0,4.0,23.0,07,26.7,2018.0,1.0,1.0,1.0,4.0,20.0,20,22.9,23,29.5,2018.0,1.0,1.0,1.0,4.0,31.0
1,Bangalore,18432984.0,Koramangala,12.932813,77.614281,BTM Layout,12.908013,77.612200,596.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.061,2.769349,CANCELLED,2018,1,1,1,4,17,2018.0,1.0,1.0,1.0,4.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df = train_data['ppdriver']
adddate_driver(df)
df.head(2)

date
login_time
logout_time


,delivery_driver_id,locality_name,city_name,latitude,longitude,date_year,date_month,date_week,date_day,login_year,login_month,login_week,login_day,login_hrs,login_min,logout_year,logout_month,logout_week,logout_day,logout_hrs,logout_min
0,28,Koramangala,Bangalore,NaN,NaN,2017,12,52,31,2018,1,1,1,5,24,2018,1,1,1,5,24
1,28,Koramangala,Bangalore,NaN,NaN,2017,12,52,31,2018,1,1,1,5,29,2018,1,1,1,5,56


In [81]:
df = train_data['prestock']
adddate_pp(df)
df.head()

time_stamp_utc
dt


,geography,Latitude,Longitude,res_id,stockout,time_stamp_year,time_stamp_month,time_stamp_week,time_stamp_day,time_stamp_hrs,time_stamp_min,time_stamp_sec,dt_year,dt_month,dt_week,dt_day
0,koramangala,12.952484,77.6228,18460060,1,2018,1,1,3,15,15,12,2018,1,1,3
1,koramangala,12.952484,77.6228,18460060,1,2018,1,1,3,15,15,39,2018,1,1,3
2,koramangala,12.952484,77.6228,18460060,1,2018,1,1,3,15,18,18,2018,1,1,3
3,koramangala,12.952484,77.6228,18460060,1,2018,1,1,3,15,18,45,2018,1,1,3
4,koramangala,12.952484,77.6228,18460060,1,2018,1,1,3,15,19,15,2018,1,1,3


In [82]:
df = train_data['poststock']
adddate_pp(df)
df.head(2)

time_stamp_utc
dt


,geography,Latitude,Longitude,res_id,stockout,time_stamp_year,time_stamp_month,time_stamp_week,time_stamp_day,time_stamp_hrs,time_stamp_min,time_stamp_sec,dt_year,dt_month,dt_week,dt_day
0,koramangala,12.952484,77.6228,18460060,0,2018,1,5,29,13,00,24,2018,1,5,29
1,koramangala,12.952484,77.6228,18460060,0,2018,1,5,29,13,00,48,2018,1,5,29


# Preprocessing Data

In this step we will look at all NAN and fill or delete the index.

In [83]:
train_data['pporder'].describe(include='all')
# print(train_data['pporder'].isnull().sum())

,city,res_id,pickup_locality,pickup_latitude,pickup_longitude,drop_locality,drop_latitude,drop_longitude,total_amount,driver_id,device_acknowledge_latitude,device_acknowledge_longitude,driver_assigned_latitude,driver_assigned_longitude,reached_shop_latitude,reached_shop_longitude,shipped_latitude,shipped_longitude,reached_customer_latitude,reached_customer_longitude,complete_latitude,pickup_to_drop_location_km,pickup_to_drop_location_aerial_km,state,created_year,created_month,created_week,created_day,created_hrs,created_min,shipment_year,shipment_month,shipment_week,shipment_day,shipment_hrs,shipment_min,device_year,device_month,device_week,device_day,device_hrs,device_min,device_acknowledge_min,device_acknowledge_sec,driver_year,driver_month,driver_week,driver_day,driver_hrs,driver_min,driver_assigned_min,driver_assigned_sec,reached_year,reached_month,reached_week,reached_day,reached_hrs,reached_min,reached_shop_min,reached_shop_sec,shipped_year,shipped_month,shipped_week,shipped_day,shipped_hrs,shipped_min,shipped_location_min,shipped_location_sec,reached_customer_min,reached_customer_sec,complete_year,complete_month,complete_week,complete_day,complete_hrs,complete_min
count,32629,3.244800e+04,32629,32629.000000,32629.000000,32629,32093.000000,32093.000000,32629.000000,31794.000000,31672.000000,31672.000000,31727.000000,31727.000000,31119.000000,31119.000000,30954.000000,30954.000000,30939.000000,30939.000000,31339.000000,32627.000000,31991.000000,32628,32629.0,32629.0,32629.000000,32629.00000,32629.000000,32629.000000,32628.0,32628.0,32628.000000,32628.000000,32628.000000,32628.000000,31672.0,31672.0,31672.000000,31672.000000,31672.000000,31672.000000,31672,31672,31727.0,31727.0,31727.000000,31727.000000,31727.000000,31727.000000,31727,31727,30939.0,30939.0,30939.000000,30939.000000,30939.000000,30939.000000,31119,31119,30954.0,30954.0,30954.000000,30954.000000,30954.000000,30954.000000,30954,30954,30939,30939,31339.0,31339.0,31339.000000,31339.000000,31339.000000,31339.000000
unique,1,NaN,2,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86,625,NaN,NaN,NaN,NaN,NaN,NaN,87,618,NaN,NaN,NaN,NaN,NaN,NaN,94,628,NaN,NaN,NaN,NaN,NaN,NaN,88,625,94,627,NaN,NaN,NaN,NaN,NaN,NaN
top,Bangalore,NaN,Koramangala,NaN,NaN,Koramangala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43,23.2,NaN,NaN,NaN,NaN,NaN,NaN,43,34.9,NaN,NaN,NaN,NaN,NaN,NaN,53,34.9,NaN,NaN,NaN,NaN,NaN,NaN,31,34.9,30,25.5,NaN,NaN,NaN,NaN,NaN,NaN
freq,32629,NaN,18279,NaN,NaN,11872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,584,80,NaN,NaN,NaN,NaN,NaN,NaN,598,106,NaN,NaN,NaN,NaN,NaN,NaN,561,95,NaN,NaN,NaN,NaN,NaN,NaN,570,89,672,75,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,9.272337e+06,NaN,12.925194,77.630946,NaN,12.924964,77.631879,354.586617,18599.778480,12.925162,77.630960,12.923371,77.626561,12.925219,77.630953,12.926327,77.631079,12.925375,77.631980,12.925358,3.111951,1.999611,NaN,2018.0,1.0,2.786294,16.69236,11.269944,29.763217,2018.0,1.0,2.786349,16.692779,11.285491,29.779974,2018.0,1.0,2.786152,16.702513,11.334144,29.726825,NaN,NaN,2018.0,1.0,2.785861,16.701327,11.337158,29.698017,NaN,NaN,2018.0,1.0,2.786806,16.707974,11.935971,29.516953,NaN,NaN,2018.0,1.0,2.787136,16.710280,11.633456,29.346159,NaN,NaN,NaN,NaN,2018.0,1.0,2.787326,16.711159,11.969367,29.532723
std,NaN,9.184851e+06,NaN,0.011133,0.012528,NaN,0.192668,0.046569,284.871266,6158.339464,0.011217,0.012648,0.103570,0.616512,0.021887,0.021590,0.143039,0.044663,0.211575,0.047357,0.210233,26.517323,21.872197,NaN,0.0,0.0,1.280875,8.80874,3.834732,17.381255,0.0,0.0,1.280856,8.808549,3.833181,17.376293,0.0,0.0,1.279591,8.788749,3.821152,17.393492,NaN,NaN,0.0,0.0,1.279153,8.784417,3.820338,17.399149,NaN,NaN,0.0,0.0,1.281299,

In [84]:
train_data['ppdriver'].describe(include='all')
# print(train_data['ppdriver'].isnull().sum())

,delivery_driver_id,locality_name,city_name,latitude,longitude,date_year,date_month,date_week,date_day,login_year,login_month,login_week,login_day,login_hrs,login_min,logout_year,logout_month,logout_week,logout_day,logout_hrs,logout_min
count,9994.000000,9994,9994,7512.000000,7512.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.0,9994.0,9994.000000,9994.000000,9994.000000,9994.000000,9994.0,9994.0,9994.000000,9994.000000,9994.000000,9994.000000
unique,NaN,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Koramangala,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,5342,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,18491.075245,NaN,NaN,12.926045,77.631900,2017.995197,1.052832,2.993296,16.208225,2018.0,1.0,2.771363,16.265459,8.775465,29.532319,2018.0,1.0,2.771363,16.265759,12.700820,31.027316
std,6337.315627,NaN,NaN,0.014726,0.011486,0.069140,0.760536,3.640733,8.910227,0.0,0.0,1.298783,8.900434,4.106363,14.008432,0.0,0.0,1.298783,8.900150,4.322108,14.885868
min,28.000000,NaN,NaN,12.899436,77.605408,2017.000000,1.000000,1.000000,1.000000,2018.0,1.0,1.000000,1.000000,0.000000,0.000000,2018.0,1.0,1.000000,1.000000,0.000000,0.000000
25%,16491.000000,NaN,NaN,12.912551,77.620886,2018.000000,1.000000,2.000000,9.000000,2018.0,1.0,2.000000,9.000000,5.000000,22.000000,2018.0,1.0,2.000000,9.000000,10.000000,22.000000
50%,20108.000000,NaN,NaN,12.925772,77.632477,2018.000000,1.000000,3.000000,16.000000,2018.0,1.0,3.000000,16.000000,8.000000,29.000000,2018.0,1.0,3.000000,16.000000,12.000000,31.000000
75%,22898.000000,NaN,NaN,12.938492,77.639832,2018.000000,1.000000,4.000000,24.000000,2018.0,1.0,4.000000,24.000000,12.000000,37.750000,2018.0,1.0,4.000000,24.000000,17.000000,41.000000


In [85]:
train_data['prestock'].describe(include='all')
# print(train_data['prestock'].isnull().sum())

,geography,Latitude,Longitude,res_id,stockout,time_stamp_year,time_stamp_month,time_stamp_week,time_stamp_day,time_stamp_hrs,time_stamp_min,time_stamp_sec,dt_year,dt_month,dt_week,dt_day
count,42384,42384.000000,42384.000000,4.238400e+04,42384.0,42384.0,42384.0,42384.000000,42384.000000,42384,42384,42384,42384.0,42384.0,42384.000000,42384.000000
unique,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,60,58,NaN,NaN,NaN,NaN
top,koramangala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,32,47,NaN,NaN,NaN,NaN
freq,27751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20195,1189,1773,NaN,NaN,NaN,NaN
mean,NaN,12.926202,77.631170,1.615524e+07,1.0,2018.0,1.0,2.474401,14.351548,NaN,NaN,NaN,2018.0,1.0,2.474401,14.351548
std,NaN,0.013128,0.009187,6.175736e+06,0.0,0.0,0.0,1.143402,8.927606,NaN,NaN,NaN,0.0,0.0,1.143402,8.927606
min,NaN,12.897420,77.613507,5.004500e+04,1.0,2018.0,1.0,1.000000,1.000000,NaN,NaN,NaN,2018.0,1.0,1.000000,1.000000
25%,NaN,12.917146,77.624418,1.841986e+07,1.0,2018.0,1.0,1.000000,6.000000,NaN,NaN,NaN,2018.0,1.0,1.000000,6.000000
50%,NaN,12.925293,77.632236,1.854398e+07,1.0,2018.0,1.0,3.000000,15.000000,NaN,NaN,NaN,2018.0,1.0,3.000000,15.000000
75%,NaN,12.937166,77.635165,1.858622e+07,1.0,2018.0,1.0,3.000000,21.000000,NaN,NaN,NaN,2018.0,1.0,3.000000,21.000000


In [86]:
train_data['poststock'].describe(include='all')

,geography,Latitude,Longitude,res_id,stockout,time_stamp_year,time_stamp_month,time_stamp_week,time_stamp_day,time_stamp_hrs,time_stamp_min,time_stamp_sec,dt_year,dt_month,dt_week,dt_day
count,61902,61902.000000,61902.000000,6.190200e+04,61902.000000,61902.0,61902.0,61902.0,61902.000000,61902,61902,61902,61902.0,61902.0,61902.0,61902.000000
unique,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,60,55,NaN,NaN,NaN,NaN
top,koramangala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,34,49,NaN,NaN,NaN,NaN
freq,33776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17376,1460,3234,NaN,NaN,NaN,NaN
mean,NaN,12.923180,77.633613,1.695352e+07,0.088123,2018.0,1.0,5.0,30.052373,NaN,NaN,NaN,2018.0,1.0,5.0,30.052373
std,NaN,0.013093,0.010463,5.197385e+06,0.283476,0.0,0.0,0.0,0.817375,NaN,NaN,NaN,0.0,0.0,0.0,0.817375
min,NaN,12.897420,77.613507,5.001300e+04,0.000000,2018.0,1.0,5.0,29.000000,NaN,NaN,NaN,2018.0,1.0,5.0,29.000000
25%,NaN,12.912649,77.626115,1.849668e+07,0.000000,2018.0,1.0,5.0,29.000000,NaN,NaN,NaN,2018.0,1.0,5.0,29.000000
50%,NaN,12.921907,77.633219,1.857330e+07,0.000000,2018.0,1.0,5.0,30.000000,NaN,NaN,NaN,2018.0,1.0,5.0,30.000000
75%,NaN,12.933433,77.638698,1.860900e+07,0.000000,2018.0,1.0,5.0,31.000000,NaN,NaN,NaN,2018.0,1.0,5.0,31.000000


# Combining datasets

In this step we will analyse relationship between various data and do concatenation if necessary to form one large dataset.

#  Baseline Model

We will first use XGBoost Model as it handles NaN cases internally.

In [95]:
train_data['poststock'].stockout.value_counts()

0    56447
1     5455
Name: stockout, dtype: int64

In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
all_data = pd.concat([train_data, test_data])

In [ ]:
drop_features = ['cat_var_1', 'cat_var_3', 'cat_var_8', 'target']
all_X = all_data.drop(drop_features, axis=1)
all_Y = all_data['target']
print (all_X.shape)
print (all_Y.shape)

In [ ]:
print(all_X.dtypes)
le = LabelEncoder()
transform = ['transaction_id', 'cat_var_2', 'cat_var_4','cat_var_5','cat_var_6','cat_var_7', 'cat_var_9','cat_var_10',\
             'cat_var_11','cat_var_12','cat_var_13','cat_var_14','cat_var_15','cat_var_16','cat_var_17','cat_var_18']
for c in transform:
    all_X[c] = le.fit_transform(all_X[c])
all_X.head(1)

In [ ]:
train_X = all_X[:348978]
train_Y = all_Y[:348978]
test_X = all_X[348978:]
print ('Training', train_X.shape, train_Y.shape)
print ('Testing', test_X.shape)

In [ ]:
print (train_Y.value_counts())
plt.bar(np.arange(2), (train_Y.value_counts()[0], train_Y.value_counts()[1]), align='center')
plt.xticks(np.arange(2), (0, 1))
plt.show()

In [ ]:
No_of_frauds= len(train_data[train_data["target"]==1])
No_of_normals = len(train_data[train_data["target"]==0])
print("The number of fraudulent transactions( Class 1) are: ", No_of_frauds)
print("The number of normal transactions( Class 0) are: ", No_of_normals)
total= No_of_frauds + No_of_normals
Fraud_percent= (float(No_of_frauds) / total)*100
Normal_percent= (float(No_of_normals) / total)*100
print("Class 0 percentage = ", Normal_percent)
print("Class 1 percentage = ", Fraud_percent)

In [ ]:
#list of fraud indices
fraud_index= np.array(train_data[train_data["target"]==1].index)

#getting the list of normal indices from the full dataset
normal_index= train_data[train_data["target"]==0].index

#choosing random normal indices equal to the number of fraudulent transactions
random_normal_indices= np.random.choice(normal_index, No_of_frauds, replace= False)
random_normal_indices= np.array(random_normal_indices)

# concatenate fraud index and normal index to create a list of indices
undersampled_indices= np.concatenate([fraud_index, random_normal_indices])

#use the undersampled indices to build the undersampled_data dataframe
undersampled_data= train_data.iloc[undersampled_indices, :]

print(undersampled_data.head())

In [ ]:
No_of_frauds_sampled= len(undersampled_data[undersampled_data["target"]== 1])

No_of_normals_sampled = len(undersampled_data[undersampled_data["target"]== 0])

print("The number of fraudulent transactions( Class 1) are: ", No_of_frauds_sampled)
print("The number of normal transactions( Class 0) are: ", No_of_normals_sampled)
total_sampled= No_of_frauds_sampled + No_of_normals_sampled
print("The total number of rows of both classes are: ", total_sampled)

Fraud_percent_sampled= (float(No_of_frauds_sampled) / total_sampled)*100
Normal_percent_sampled= (float(No_of_normals_sampled) / total_sampled)*100
print("Class 0 percentage = ", Normal_percent_sampled)
print("Class 1 percentage = ", Fraud_percent_sampled)

#Check the data count now
count_sampled=pd.value_counts(undersampled_data["target"], sort= True)
count_sampled.plot(kind= 'bar')

In [ ]:
drop_features = ['cat_var_1', 'cat_var_3', 'cat_var_8']
undersampled_data = undersampled_data.drop(drop_features, axis=1)
print (undersampled_data.shape)

In [ ]:
le = LabelEncoder()
transform = ['transaction_id', 'cat_var_2', 'cat_var_4','cat_var_5','cat_var_6','cat_var_7', 'cat_var_9','cat_var_10',\
             'cat_var_11','cat_var_12','cat_var_13','cat_var_14','cat_var_15','cat_var_16','cat_var_17','cat_var_18']
for c in transform:
    undersampled_data[c] = le.fit_transform(undersampled_data[c])
undersampled_data.head(1)

In [ ]:
# sc = StandardScaler()
# sscale = ['cat_var_2', 'cat_var_4','cat_var_5','cat_var_6','cat_var_7', 'cat_var_9','cat_var_10','cat_var_11',\
#           'cat_var_12','cat_var_13','cat_var_14','cat_var_15','cat_var_16','cat_var_17','cat_var_18']
# for c in sscale:
#     undersampled_data[c] = sc.fit_transform(undersampled_data[c])
# undersampled_data.head(1)

In [ ]:
X= undersampled_data.iloc[:, undersampled_data.columns != "target"].values
y= undersampled_data.iloc[:, undersampled_data.columns == "target"].values
print ('Traning', X.shape, y.shape)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_X, train_Y, test_size=0.25, random_state=7)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def selectKImportance(model, X, k=5):
     return X[:,model.feature_importances_.argsort()[::-1][:k]]
    

In [ ]:
def printing_Kfold_scores(train_x, train_y):
    fold = KFold(len(train_y),5,shuffle=False) 

    # Different C parameters
    c_param_range = [0.01,0.1,1,10,100]

    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # the k-fold will give 2 lists: train_indices = indices[0], test_indices = indices[1]
    j = 0
    for c_param in c_param_range:
        print('-------------------------------------------')
        print('C parameter: ', c_param)
        print('-------------------------------------------')
        print('')

        recall_accs = []
        for iteration, indices in enumerate(fold,start=1):

            # Call the logistic regression model with a certain C parameter
            lr = LogisticRegression(C = c_param, penalty = 'l1')

            # Use the training data to fit the model. In this case, we use the portion of the fold to train the model
            # with indices[0]. We then predict on the portion assigned as the 'test cross validation' with indices[1]
            lr.fit(train_x, train_y.ravel())

            # Predict values using the test indices in the training data
            y_pred_undersample = lr.predict(val_x)

            # Calculate the recall score and append it to a list for recall scores representing the current c_parameter
            recall_acc = recall_score(val_y.ravel(), y_pred_undersample)
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': recall score = ', recall_acc)

        # The mean value of those recall scores is the metric we want to save and get hold of.
        results_table.ix[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Mean recall score ', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].idxmax()]['C_parameter']
    
    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Best model to choose from cross validation is with C parameter = ', best_c)
    print('*********************************************************************************')
    
    return best_c

best_c = printing_Kfold_scores(train_x, train_y)

In [ ]:
# Use this C_parameter to build the final model with the whole training dataset and predict the classes in the test
# dataset
lr = LogisticRegression(C = best_c, penalty = 'l1')
lr.fit(train_x, train_y.ravel())
y_pred_undersample = lr.predict(val_x)

# Compute confusion matrix
cnf_matrix = confusion_matrix(val_y, y_pred_undersample)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", float(cnf_matrix[1,1])/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()
print ('Score:', roc_auc_score(val_y, y_pred_undersample))

In [ ]:
model = xgb.XGBClassifier()
model.fit(train_x, train_y.ravel())

# # Fit model using each importance as a threshold
# thresholds = np.sort(model.feature_importances_)
# thresholds = thresholds[::-1]
# print (thresholds, len(thresholds))
# for thresh in thresholds:
#     # select features using threshold
#     selection = SelectFromModel(model, threshold=thresh, prefit=True)
#     select_X_train = selection.transform(train_x)
#     # train model
#     selection_model = xgb.XGBClassifier()
#     selection_model.fit(select_X_train, train_y)
#     # eval model
#     select_X_test = selection.transform(val_x)
#     y_pred = selection_model.predict(select_X_test)
#     predictions = [round(value) for value in y_pred]
#     roc_score =roc_auc_score(val_y, predictions)
#     print("Thresh=%.3f, n=%d, Roc: %.2f" % (thresh, select_X_train.shape[1], roc_score))

# new_x = selectKImportance(model, train_x, k=20)
# new_val_x = selectKImportance(model, val_x, k=20)
model = xgb.XGBClassifier(max_depth=7, n_estimators=500)
print (model.fit(train_x, train_y.ravel()))
# plot_feature_importances(model, max_num_features=20, x_tick_rotation=-90, feature_names=undersampled_data.columns)
# plt.show()
pred_class_y = model.predict(val_x)
pred_val_y = model.predict_proba(val_x)
print (pred_class_y.shape, pred_val_y.shape, val_x.shape, val_y.shape)
cm = confusion_matrix(val_y, pred_class_y)
plot_confusion_matrix(cm, [0, 1])
plt.show()
print ('AUC-ROC:')
plot_roc_curve(val_y, pred_val_y)
plt.show()
print ('Score:', roc_auc_score(val_y, pred_class_y))

#Applying 10 fold cross validation
# rc_score = cross_val_score(estimator = model, X=train_x, y = train_y.ravel(), cv = 10, scoring='roc_auc')
# print("The rc score accuracy in %: ", rc_score.mean())

In [ ]:
clf = RandomForestClassifier(n_estimators=500, class_weight='balanced')
print (clf.fit(train_x, train_y.ravel()))
# plot_feature_importances(clf, max_num_features=47, x_tick_rotation=-90, feature_names=undersampled_data.columns)
# plt.show()
# new_x = selectKImportance(clf, train_x, k=20)
# new_val_x = selectKImportance(clf, val_x, k=20)

# plot_feature_importances(clf, max_num_features=47, x_tick_rotation=-90, feature_names=undersampled_data.columns)
# plt.show()
# importances = clf.feature_importances_
# std = np.std([tree.feature_importances_ for tree in clf.estimators_],
#              axis=0)
# indices = np.argsort(importances)[::-1]
# # Print the feature ranking
# print("Feature ranking:")

# for f in range(X.shape[1]):
#     print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
clf.fit(train_x, train_y.ravel())
pred_class_y = clf.predict(val_x)
pred_val_y = clf.predict_proba(val_x)
print (pred_class_y.shape, pred_val_y.shape, val_x.shape, val_y.shape)
cm = confusion_matrix(val_y, pred_class_y)
plot_confusion_matrix(cm, [0, 1])
plt.show()
print ('AUC-ROC:')
plot_roc_curve(val_y, pred_val_y)
plt.show()
print ('Score:', roc_auc_score(val_y, pred_class_y))

#Applying 10 fold cross validation
# rc_score = cross_val_score(estimator = clf, X=train_x, y = train_y.ravel(), cv = 10, scoring='roc_auc')
# print("The rc score accuracy in %: ", rc_score)

In [ ]:
clf = ExtraTreesClassifier(n_estimators=100, class_weight='balanced')
print (clf.fit(train_x, train_y.ravel()))
# plot_feature_importances(clf, max_num_features=47, x_tick_rotation=-90, feature_names=undersampled_data.columns)
# plt.show()

clf.fit(train_x, train_y.ravel())
pred_class_y = clf.predict(val_x)
pred_val_y = clf.predict_proba(val_x)
print (pred_class_y.shape, pred_val_y.shape, val_x.shape, val_y.shape)
cm = confusion_matrix(val_y, pred_class_y)
plot_confusion_matrix(cm, [0, 1])
plt.show()
print ('AUC-ROC:')
plot_roc_curve(val_y, pred_val_y)
plt.show()
print ('Score:', roc_auc_score(val_y, pred_class_y))

In [ ]:
clf = BaggingClassifier(n_estimators=100)
print (clf.fit(train_x, train_y.ravel()))
pred_class_y = clf.predict(val_x)
pred_val_y = clf.predict_proba(val_x)
print (pred_class_y.shape, pred_val_y.shape, val_x.shape, val_y.shape)
cm = confusion_matrix(val_y, pred_class_y)
plot_confusion_matrix(cm, [0, 1])
plt.show()
print ('AUC-ROC:')
plot_roc_curve(val_y, pred_val_y)
plt.show()
print ('Score:', roc_auc_score(val_y, pred_class_y))

In [ ]:
clf = AdaBoostClassifier()
print (clf.fit(train_x, train_y.ravel()))
pred_class_y = clf.predict(val_x)
pred_val_y = clf.predict_proba(val_x)
print (pred_class_y.shape, pred_val_y.shape, val_x.shape, val_y.shape)
cm = confusion_matrix(val_y, pred_class_y)
plot_confusion_matrix(cm, [0, 1])
plt.show()
print ('AUC-ROC:')
plot_roc_curve(val_y, pred_val_y)
plt.show()
print ('Score:', roc_auc_score(val_y, pred_class_y))

#Applying 10 fold cross validation
# rc_score = cross_val_score(estimator = clf, X=train_x, y = train_y.ravel(), cv = 10, scoring='roc_auc')
# print("The rc score accuracy in %: ", rc_score)

In [ ]:
# clf= SVC(C= 1, kernel= 'rbf', random_state= 0)
# clf.fit(train_x, train_y.ravel())
# y_pred = classifier.predict(val_x)
# pred_class_y = clf.predict(val_x)
# pred_val_y = clf.predict_proba(val_x)
# print (pred_class_y.shape, pred_val_y.shape, val_x.shape, val_y.shape)
# cm = confusion_matrix(val_y, pred_class_y)
# plot_confusion_matrix(cm, [0, 1])
# plt.show()
# print ('AUC-ROC:')
# plot_roc_curve(val_y, pred_val_y)
# plt.show()
# print ('Score:', roc_auc_score(val_y, pred_class_y))

In [ ]:
from mlxtend.classifier import StackingClassifier

# clf1 = ExtraTreesClassifier(random_state = 45, n_estimators = 100)
clf2 = RandomForestClassifier(random_state = 45, n_estimators = 500)
clf3 = xgb.XGBClassifier(seed = 45, learning_rate = 0.1, n_estimators = 500, max_depth = 5)
clf4 = BaggingClassifier(n_estimators = 100)
# print ('ExtraTree:', cv_model(clf1))
# print ('RF:', cv_model(clf2))
# print ('XGB:', cv_model(clf3))
# print ('BaggingTree:', cv_model(clf4))
# Compute stacking features
model =  StackingClassifier(classifiers=[clf2, clf4], meta_classifier=clf3, verbose=1)
# print ('Stack:', cv_model(model))
model.fit(train_x, train_y)

# # Final prediction score
# print('Final r2 score: [%.8f]' % model.score(val_x, val_y))
# pred_test_y = model.predict(test_x)

pred_class_y = model.predict(val_x)
pred_val_y = model.predict_proba(val_x)
print (pred_class_y.shape, pred_val_y.shape, val_x.shape, val_y.shape)
cm = confusion_matrix(val_y, pred_class_y)
plot_confusion_matrix(cm, [0, 1])
plt.show()
print ('AUC-ROC:')
plot_roc_curve(val_y, pred_val_y)
plt.show()
print ('Score:', roc_auc_score(val_y, pred_class_y))

In [ ]:
test_pred = model.predict_proba(test_X)
sub = pd.DataFrame({'transaction_id':test_data['transaction_id'], 'target':test_pred[:, 1]})
sub.to_csv('submit.csv', columns=['transaction_id', 'target'], index=False)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD

def base_model():
    model = Sequential()
    model.add(Dense(20, input_dim=20, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
#     model.add(Dense(128, activation='elu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(32, activation='elu'))
#     model.add(Dropout(0.25))
    model.add(Dense(8, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['acc'])
    return model

model = base_model()
# train_X = new_x.as_matrix().astype('float32')
# train_Y = train_y.as_matrix().astype('float32')
# val_X = new_val_x.as_matrix().astype('float32')
# val_Y = val_y.as_matrix().astype('float32')
# one_hot_labels = keras.utils.to_categorical(train_Y, num_classes=2)
model.fit(new_x, train_y.ravel(), batch_size=32, epochs=10)
pred_class_Y = model.predict(new_val_x)
pred_class_Y = np.reshape(pred_class_Y, (-1,))
pred_val_Y = model.predict_proba(new_val_x)
cm = confusion_matrix(val_y.ravel(), pred_class_Y)
plot_confusion_matrix(cm, [0, 1])
plt.show()
plot_roc_curve(val_Y, pred_val_Y)
plt.show()
print ('Score:', roc_auc_score(val_Y, pred_class_Y))

In [ ]:
print (pred_class_Y.shape)
print (new_val_x.shape)
print (val_y.shape)
print (pred_val_Y)
print (pred_class_Y)
print (val_y)
print (new_x.dtype, train_y.dtype, new_val_x.dtype, val_y.dtype)
print (pred_val_Y.dtype, pred_class_Y.dtype)
print (np.reshape(pred_class_Y, (-1,)).shape)


In [ ]:
print ('AUC-ROC:')
print (pred_val_Y)
#plot_roc_curve(val_y.ravel(), pred_val_Y.ravel())
#plt.show()
print ('Score:', roc_auc_score(val_y.ravel(), pred_val_Y))

In [ ]:
pred_val_Y[pred_val_Y > 0.5] = 1
pred_val_Y[pred_val_Y < 0.5] = 0
print (np.where(pred_val_Y >= 0.5, 1, 0))
print (pred_val_Y)

In [ ]:
select = ['cat_var_2', 'transaction_id','cat_var_4','cat_var_11','cat_var_15', 'cat_var_14','cat_var_6','cat_var_13',\
          'cat_var_7','cat_var_9','cat_var_12','cat_var_10','cat_var_5','num_var_5','num_var_7','num_var_4','num_var_1',\
         'num_var_2','num_var_3','num_var_6']
new_test_x = test_X[select]
print (new_test_x[:2], new_test_x.shape)

In [ ]:
new_test_x.rename(columns={'cat_var_2':'f2', 'transaction_id':'f1','cat_var_4':'f0','cat_var_11':'f6','cat_var_15':'f9',\
                           'cat_var_14':'f10','cat_var_6':'f11','cat_var_13':'f13','cat_var_7':'f15','cat_var_9':'f16',\
                           'cat_var_12':'f17','cat_var_10':'f18','cat_var_5':'f19','num_var_5':'f3','num_var_7':'f4',\
                           'num_var_4':'f5','num_var_1':'f7','num_var_2':'f8','num_var_3':'f12','num_var_6':'f14'}, \
                  inplace=True)

In [ ]:
print (new_test_x.columns)

In [ ]:
cols = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19']
new_test_x = new_test_x[cols]
print (new_test_x.columns)